In [ ]:
from itertools import combinations
from collections import defaultdict
import numba
import pandas as pd
import json

# Hand Ratings

This notebook calculates and outputs a rating for each subset of cards in a cribbage hand (completely ignoring suit and flushes, which are going to be rare in a 5x5 grid game).

We start by finding the number of occurrences of unique hand, where we change ranks to numbers 1 through 13, with 1 = Ace and 13 = King.


In [ ]:
deck = []
for i in range(1, 14):
    deck.extend(4 * [i])



hand_frequency = defaultdict(int)

for hand in combinations(deck, 5):
    hand = tuple(sorted(hand))
    hand_frequency[hand] += 1

# Finding Consistent Hands with a Subset of Cards

We make a couple functions to find hands that are consistent with a given subset of cards. For example the hands consistent with (1,2,3,4) are 13 in number, (1,2,3,4) and any number between 1 and 13  as the 5th card.

In [ ]:

@numba.jit(nopython=True)
def tuple_subset(x, y):
    y = list(y)
    for elem in x:
        if elem in y:
            y.remove(elem)
        else:
            return False
    return True
    

In [ ]:
def get_consistent_hands(cards):
    return filter(lambda key: tuple_subset(cards, key), hand_frequency.keys())

a = get_consistent_hands((1,2,3,4))
list(a)

# Cribbage Scoring

We create a function scoreHand() to calculate the score for a 5 card hand based on pairs, 15s, and runs. We ignore flushes here. We then score every unique hand.

In [ ]:
def scorePairs(hand):
    if len(hand) < 2:
        return 0
    return sum(map(lambda x: 1 if x[0]==x[1] else 0, combinations(hand,2)))*2


from itertools import chain, combinations

def powerset2(iterable):
    """
    all subsets with >= 2 elements
    powerset2([1,2,3]) --> (1,2) (1,3) (2,3) (1,2,3)
    """
    xs = list(iterable)
    if len(xs) < 2:
        return []
    # note we return an iterator rather than a list
    return chain.from_iterable(combinations(xs,n) for n in range(len(xs)+1))


def score15(hand):
    if len(hand) < 2:
        return 0
    hand = map(lambda x: min(10, x), hand)
    return sum(map(lambda x: 2 if sum(x)==15 else 0, powerset2(hand)))


def scoreRuns(hand):
    numbers = list(sorted(hand))
    numbers.append(1000)
    
    score = 0
    duplicity = 1
    multiple = 1
    currentLength = 1
    for i in range(1, len(numbers)):
        current = numbers[i]
        prev = numbers[i-1]
        delta = current - prev
        if delta == 0:
            duplicity += 1
        elif delta == 1:
            multiple *= duplicity;
            duplicity = 1;
            currentLength += 1;
        else:
            if currentLength > 2:
                score += (currentLength * multiple * duplicity)
            currentLength = 1;
            duplicity = 1;
            multiple = 1;
    return score


def scoreHand(hand):
    score = 0
    score += score15(hand)
    score += scorePairs(hand)
    score += scoreRuns(hand)

    return score


hand_scores = {}
for hand in hand_frequency.keys():
    hand_scores[hand] = scoreHand(hand)

# Getting Card Ratings

For every subset of cards of size n (n=1,2,3,4), we calculate the conditional expected value of points for that subset assuming we fill out the rest of the hand with random cards. For example, if we have the 4 card subset (1,1,1,1), there are 12 unique consistent hands (1,1,1,1,2), (1,1,1,1,3), ....., (1,1,1,1,13), each with equal probability. The rating for (1,2,3,4) is then the average of the point values for each of these 12 hands.

In [ ]:
def get_n_card_ratings(n):
    card_ratings = {}
    for cards in combinations(deck, n):
        cards = tuple(sorted(cards))
        if cards in card_ratings:
            continue
        numerator = 0
        denominator = 0
        for hand, count in hand_frequency.items():
            if tuple_subset(cards, hand):
                numerator += (count * hand_scores[hand])
                denominator += count
        card_ratings[cards] = numerator/denominator
    return card_ratings

In [ ]:
def get_rating_frame(n):
    combo, avg_scores = zip(*get_n_card_ratings(n).items())
    return pd.DataFrame({"cards": combo, "score": avg_scores, "num": [n]*len(avg_scores)})

df = pd.concat([get_rating_frame(i) for i in range(1, 5)])

# Encode and Output for Use in JS
We convert each hand to a handId by assuming that the rank entries encode a base 14 number. This will be sparse of course. We output a JSON file that maps each handId to its rating (this includes all 1, 2, 3, and 4 card subsets). We don't need to do 5 card subsets because they are complete hands with a fixed exact score. 

In [ ]:
def convert_tuple_to_number(tup):
    ans = 0
    for entry in tup:
        ans *= 14
        ans += entry
    return ans
df['handId'] = df.cards.apply(convert_tuple_to_number)

In [ ]:
output_dict = dict(zip(df.handId, df.score))
with open('public/ratings.json', 'w') as f:
    json.dump(output_dict, f)